In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.0'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Get data.
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [4]:
# Convert data into dataframe.
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [5]:
# 1. Filter the data and create a new table to retrieve all rows where total_votes >= 20.
df_high_votes = df.filter("total_votes>=20")

In [6]:
# 2. Filter the DF from Step 1 and create a new DF to retrieve all rows where the number of helpful_votes divided by total_votes is equal or greater than 50%.

# 2a. Create a new column with helpful_votes / total_votes.
df_high_votes2 = df_high_votes.withColumn('50perc_helpful_votes', df_high_votes['helpful_votes']/df_high_votes['total_votes'])

In [7]:
# 2b. Filter for >= 50% of helpful votes.
df_helpful_votes = df_high_votes2.filter("50perc_helpful_votes>.5")

In [8]:
# 3. Filter the DF from Step 2 for all reviews with Vine program paid.
df_vine = df_helpful_votes.filter(df_helpful_votes["marketplace"] == "US")

In [9]:
# 4. Filter the DF from Step 2 for all reviews outside the Vine program. 
df_not_vine = df_helpful_votes.filter(df_helpful_votes["vine"] == "N")

In [ ]:
# 5. Determine total num of reviews, 5-star reviews, and perc of 5-star reviews for both types.
df_vine.count()

In [15]:
# 5a. Vine - num reviews.
df_vine.count()

61807

In [21]:
# 5b. Vine - 5-star reviews.
df_vine.filter("star_rating > 4").count()

32785

In [22]:
# 5c. Vine - Perc of 5-star reviews.
32785/61807

0.530441535748378

In [23]:
# 5d. Not vine - num reviews.
df_not_vine.count()

61474

In [24]:
# 5e. Not vine - 5-star reviews.
df_not_vine.filter("star_rating > 4").count()

32646

In [25]:
# 5f. Not vine - Perc of 5-star reviews.
32646/61474


0.5310537788333279